<a href="https://colab.research.google.com/github/Valelugo/Business-Inteligence/blob/main/Reto_%C3%A1rbol_decisi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Análisis de modelo empleando el árbol de decisión**

Se analiza un modelo para determinar la influencia de las variables socioeconómicas y demográficas en la admisión a la universidad.


0. Descarga librerías de trabajo

In [ ]:
#Librerías científicas
import numpy as np
import pandas as pd

#librerías arbol
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Conectar al drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Carga base de datos y explicación variables

In [ ]:
nxl='/content/drive/MyDrive/2. CollegeAdmisions_Int_M.xlsx'
XDB=pd.read_excel(nxl, sheet_name=0) #Información estudiantes
XDB.head(100)
XDB=XDB.dropna()


#Seleccionar las variables
XD=XDB[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']] #Variable de entrada
yd=XDB[['Admitted']] #Variable de salida

#Convertir Gender en una variable numérica using .loc
XD.loc[:, 'Gender'] = XD['Gender'].map({'F': 0, 'M': 1}) #El Loc es para no confundir al data frame con que es una nueva base o una copia. Indica modificaciones en la misma base de datos.




,Edu_Parent1,Edu_Parent2,Gender,White,Asian,HSGPA,SAT/ACT
6,6,6,0,1,0,4.26,1430
28,7,6,1,1,0,3.95,1340
46,3,6,1,1,0,4.33,1200
61,7,7,0,1,0,4.27,1530
65,6,7,0,0,0,4.04,1270
...,...,...,...,...,...,...,...
17330,2,5,1,0,0,2.80,1010
17332,7,4,1,1,0,3.04,1130
17333,4,6,1,1,0,2.80,1210
17334,7,6,1,0,0,3.42,1000


**Descripción de los datos**
De acuerdo con la base de datos de estudiantes, la cual posee un total de 1528 datos dividios en 1303 admitidos y 225 no admitidos. Cada uno de los posibles candidatos está descrito en función de 7 variables:

* Edu_parent: indica el número de años de educación de los padres, en función a las categorías del nivel educativo. (Secundaria, bachillerato, universidad y posgrado)
* Gender: se trata de una variable categórica que indica si la persona admitida es hombre o mujer.
* White: se refiere a una variable racial que indica si el individuo es blanco.
* Asian: indica una variable étnica que indica si el estudiante se identifica como asiático.
* HSGPA: se trata de una medida que representa el desempeño académico del estudiante a lo largo del bachillerato.
* SAT/ACT: enlista la puntuación puntuación promedio del exámen SAT que evalúa la escritura y las matemáticas.
* Admitted: es un proceso competitivo, en el que se evalúa al estudiante para determinar si es o no admitido en la universidad.



Definir las variables utilizadas

2. Se procede con la implementación del modelo

In [ ]:
mar=DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=42)
mar.fit(XD,yd)

#Se procede con la gráfica
from pydotplus import graph_from_dot_data #Crea la gráfica
from sklearn.tree import export_graphviz #Lo exporta a PNG

ve=['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']
dot_data=export_graphviz(mar, feature_names=ve) #Crear el gráfico
graph=graph_from_dot_data(dot_data)
graph.write_png('ArbolDecision.png')

True

3. Se procede con el pronóstico de estudiantes que no fueron admitidos.

In [ ]:
# Conteo de estudiantes admitidos
admitted_count = XDB['Admitted'].value_counts()['Yes']

# Conteo de estudiantes no admitidos
not_admitted_count = XDB['Admitted'].value_counts()['No']

print("Estudiantes admitidos:", admitted_count)
print("Estudiantes no admitidos:", not_admitted_count)


Estudiantes admitidos: 1303
Estudiantes no admitidos: 225


4. Determinar el número de nodos puros (Aunque se pueden contar manualmente en el pdf del árbol, gemini ayudó a crear un código para determinarlos)

In [ ]:
# Considerar la estructura del árbol
tree = mar.tree_

# Conteo de los nodos puros
pure_nodes_count = 0

for node_id in range(tree.node_count):
    # Comprueba si el nodo es un nodo hoja, es decir, un nodo que no tiene hijos y representa una decisión final o una predicción.
    if tree.children_left[node_id] == tree.children_right[node_id]:
        # Comprueba si es un nodo puro(all samples belong to the same class)
        if tree.impurity[node_id] == 0:
            pure_nodes_count += 1

# Print the number of pure nodes
print("Number of pure nodes:", pure_nodes_count)

Number of pure nodes: 7


5. Llevar a cabo el pronóstico de la admisión para las personas agrupadas en la base de datos del pronóstico (sheet = 1)

In [ ]:
# Load the forecast database (sheet 1)
datos = pd.read_excel('/content/drive/MyDrive/2. CollegeAdmisions_Int_M.xlsx', sheet_name=1)

Pronosticar_datos = datos

#Reseleccionar las variables
XD=XDB[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']] #Variable de entrada
yd=XDB[['Admitted']] #Variable de salida

#Convertir Gender en una variable numérica using .loc
XD.loc[:, 'Gender'] = XD['Gender'].map({'F': 0, 'M': 1})

# Crear un subconjunto de los datos de pronóstico con características relevantes
X_pronostico = Pronosticar_datos[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']]

# Convertir Gender en una variable numérica
X_pronostico.loc[:, 'Gender'] = X_pronostico['Gender'].map({'F': 0, 'M': 1})


predicciones = mar.predict(X_pronostico)
Pronosticar_datos['AdmissionPrediction'] = predicciones
Pronosticar_datos.to_excel('pronosticoestudiantes.xlsx', index=False)

6. Evaluar el comportamiento del modelo

In [ ]:
ydp=mar.predict(XD) #Pronóstico sobre la base de datos original
cm=confusion_matrix(yd, ydp)
print("La matriz de confusión es: \n", cm)

VN=cm[0,0]; FP=cm[0,1];FN=cm[1,0]; VP=cm[1,1]

Exactitud=(VP+VN)/(VP+VN+FP+FN)
Sensibilidad=VP/(VP+FN)
Especificidad=VN/(VN+FP)
Precision=VP/(VP+FP)
TasaError=(FP+FN)/(VP+VN+FP+FN)
PredNeg=VN/(VN+FN)

print("La exactitud es: ", Exactitud)
print("La sensibilidad es: ", Sensibilidad)
print("La especificidad es: ", Especificidad)
print("La precision es: ", Precision)
print("La tasa de error es: ", TasaError)
print("La presición negativa es:", PredNeg)

La matriz de confusión es: 
 [[  20  205]
 [   2 1301]]
La exactitud es:  0.8645287958115183
La sensibilidad es:  0.9984650805832693
La especificidad es:  0.08888888888888889
La precision es:  0.8638778220451527
La tasa de error es:  0.1354712041884817
La presición negativa es: 0.9090909090909091


**Análisis de resultados**
La matriz de confusión muestra una predominancia por los valores Verdaderos Positivos, equivalentes a 1301 datos. Esto contribuyó a que el sistema alcanzara porcentajes superiores al 80% para los índices de exactitud, sensibilidad y precisión; además de tener una tasa de error por debajo del 15%, lo que indica que el sistema no tiene un sesgo elevado. Con respecto al número total de datos, el modelo destaca que 1303 estudiantes fueron admitidos, mientras que 225 no; cabe resaltar la información de la base de datos, la cual no carecía de ningún valor para la variable de salida, por lo que no fue necesario predecir. A su vez, se ubican 7 nodos puros con un gini equivalente al 0%, lo que indica la homogenidad y veracidad de los datos. De acuerdo con los índices, el modelo muestra un mejor comportamiento en la sensibilidad (99,8%) y la presición negativa (90,90%), a diferencia de la especificidad. (8,8%)